In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , OneHotEncoder , LabelEncoder , label_binarize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.client import device_lib
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve, average_precision_score , classification_report

In [2]:
file_path = r"C:\\Users\\Amirreza\\Downloads\\Documents\\Project\\Research paper\\Data\\real_data_2.csv"
df = pd.read_csv(file_path)
df

,timestamp,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,class
0,2013-11-04 01:41:30.500000000,0.0,1.885235e+07,116.775477,9.599386e+06,173.096100,0.0
1,2013-11-04 01:42:30.500000000,0.0,1.885235e+07,116.769880,9.599286e+06,173.096100,0.0
2,2013-11-04 01:43:30.500000256,0.0,1.885235e+07,116.764285,9.599185e+06,173.096100,0.0
3,2013-11-04 01:44:30.500000000,0.0,1.885235e+07,116.758693,9.599085e+06,173.096100,0.0
4,2013-11-04 01:45:30.500000000,0.0,1.885235e+07,116.766512,9.598985e+06,173.096100,0.0
...,...,...,...,...,...,...,...
212671,2019-04-03 14:55:35.500000000,0.0,8.490801e+06,109.750025,1.486534e+06,73.622858,107.0
212672,2019-04-03 14:56:35.500000000,0.0,8.490216e+06,109.763962,1.500514e+06,73.576691,107.0
212673,2019-04-03 14:57:35.500000000,0.0,8.490409e+06,109.769658,1.485919e+06,73.494505,107.0
212674,2019-04-03 14:58:35.500000000,0.0,8.490884e+06,109.759015,1.492781e+06,73.363835,107.0


In [3]:
def preprocess_data(df, sequence_length):
    # drop unwanted column
    # df = df.drop(columns=['P-JUS-CKGL', 'T-JUS-CKGL', 'QGL'])

    # convert timestamp to datetime and extract features
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['second'] = df['timestamp'].dt.second
    df['seconds_since_midnight'] = df['hour'] * 3600 + df['minute'] * 60 + df['second']
    df = df.drop(columns=['timestamp', 'hour', 'minute', 'second'])

    #separate features and target
    X = df.drop(columns=['class'])
    y = df['class']

    # preprocessing pipeline for numeric features
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    numeric_transformer = StandardScaler()

    # Preprocessing pipeline for categorical features
    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('car', categorical_transformer, categorical_features)
        ]
    )

    # apply transformation 
    X_processed = preprocessor.fit_transform(X)

    # encode class labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Initialize lists for sequences
    X_sequences = []
    y_sequences = []


    # Create sequences
    for i in range(len(X_processed) - sequence_length):
        X_sequences.append(X_processed[i:i + sequence_length])
        y_sequences.append(y_encoded[i + sequence_length])

    # create sequences
    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)

    return X_sequences, y_sequences

In [4]:
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=75, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=75))
    model.add(Dense(units=15, activation='softmax'))  # 15 units for 15 classes

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [5]:
# Lists to store results
acc_scores = []
val_acc_scores = []
test_accuracy = []

# Define sequence length
sequence_length = 60   # 1 hours

# preprocess the data
X, y = preprocess_data(df, sequence_length)

# Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.05/(1-0.05), random_state=42)

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

# Open a strategy scope.
with strategy.scope():
    # Create the model inside the strategy's scope.
    model = create_model(input_shape=(sequence_length, X_train.shape[2]))

    # Define the early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Evaluate on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Epoch 1/10


5980/5980 [==============================] - 289s 47ms/step - loss: 0.2599 - accuracy: 0.9139 - val_loss: 0.2204 - val_accuracy: 0.9221
Epoch 2/10
5980/5980 [==============================] - 424s 71ms/step - loss: 0.2002 - accuracy: 0.9313 - val_loss: 0.2252 - val_accuracy: 0.9243
Epoch 3/10
5980/5980 [==============================] - 302s 50ms/step - loss: 0.1987 - accuracy: 0.9303 - val_loss: 0.1951 - val_accuracy: 0.9320
Epoch 4/10
5980/5980 [==============================] - 341s 57ms/step - loss: 0.1776 - accuracy: 0.9385 - val_loss: 0.1524 - val_accuracy: 0.9494
Epoch 5/10
5980/5980 [==============================] - 362s 60ms/step - loss: 0.1690 - accuracy: 0.9412 - val_loss: 0.1767 - val_accuracy: 0.9430
Epoch 6/10
5980/5980 [==============================] - 360s 60ms/step - loss: 0.1673 - accuracy: 0.9418 - val_loss: 0.1488 - val_accuracy: 0.9480
Epoch 7/10
5

In [6]:
# Print the results
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9551312327384949


In [7]:
model.save("LSTM_final_model_for_real_data.h5")

c:\Users\Amirreza\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:

model1 = load_model("LSTM_final_model.h5")

In [ ]:
# styling for plots
sns.set(style="whitegrid")
plt.style.use('ggplot')

In [ ]:
# Confusion Matrix
def plot_confusion_matrix(y_test , y_pred, classes):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='blues', xticklabels=classes, yticklabels=classes)
    plt.title(f'LSTM_confusion_matrix')
    plt.ylabel('True label')
    plt.xlabel('predicted label')
    plt.savefig(f'LSTM_confusion_matrix.png')
    plt.show()

In [ ]:
# ROC Curve and AUC
def plot_roc_curve(y_test, y_score, n_classes):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range (n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    plt.figure(figsize=(10,7))
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label='class{0} (area = {1:0.2f})'.format(i, roc_auc))
        
    plt.plot([0,1], [0,1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'LSTM - Receivor Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig(f'LSTM_roc_curve.png')
    plt.show()

In [ ]:
# Precision-Recall Curve
def plot_precision_recall_curve(y_test, y_score, n_classes):
    precision = dict()
    recall = dict()
    average_precision = dict()
    
    for i in range (n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:,i], y_score[:,i])
        average_precision[i] = average_precision_score(y_test[:,i], y_score[:,i])
        
    plt.figure(figsize=(10,7))
    for i in range (n_classes):
        plt.plot(recall[i], precision[i], lw=2, label='Class[0] (area = {1:0.2f})'.format(i, average_precision[i]))
        
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'LSTM - precision-recall curve')
    plt.legend(loc="best")
    plt.savefig(f'LSTM_precision_recall_curve.png')
    plt.show()
        

In [2]:
# Training and Validation plots
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)
    
    # accuracy plot 
    plt.figure(figsize=(7,5))
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy')
    plt.title('LSTM - Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f'LSTM_training_accuracy')
    
    
    # Loss plot 
    plt.figure(figsize=(7,5))
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'r*-', label='Validation loss')
    plt.title('LSTM - Training and validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'LSTM_training_loss')
    plt.show()



In [ ]:
# Generate classification report
def save_classification_report(y_test, y_pred, classes):
    report = classification_report(y_test, y_pred, target_names=classes)
    print(report)
    with open(f"LSTM_classification_report.txt", "w") as f:
        f.write(report)

In [ ]:
classes = [str(i) for i in range(15)] 

# Predict on test data
y_pred = np.argmax(model1.predict(X_test), axis=-1)
y_score = model1.predict(X_test)

# Binarize the output for multiclass metrics
y_test_bin = label_binarize(y_test, classes=range(len(classes)))

# Plot and save all the metrics
plot_training_history(history)
plot_confusion_matrix(y_test, y_pred, classes)
plot_roc_curve(y_test_bin, y_score, len(classes))
plot_precision_recall_curve(y_test_bin, y_score, len(classes))
save_classification_report(y_test, y_pred, classes)